In [ ]:
import cv2
import numpy as np
import time
from collections import defaultdict
from ultralytics import YOLO
import pandas as pd

# Configuración de las líneas de referencia (verticales)
line_start_x = 500  # Coordenada X de la línea de inicio (ajusta según tu necesidad)
line_end_x = 1300    # Coordenada X de la línea de final (ajusta según tu necesidad)
line_color = (0, 255, 0)  # Color de la línea (verde)
line_thickness = 2

# Distancia conocida entre las líneas (en metros)
distance_between_lines = 22  # Cambia esto según tu escenario

# Inicializar YOLOv8
model = YOLO("yolov8n.pt")  # Usar un modelo más grande para mejor precisión

# Diccionario para almacenar el tiempo de cruce de cada vehículo
vehicle_times = defaultdict(list)

# DataFrame para almacenar los resultados finales
results_df = pd.DataFrame(columns=["ID", "Speed (km/h)", "Class"])

# Función para calcular la velocidad en km/h
def calculate_speed(distance, time_diff):
    if time_diff == 0:
        return 0
    return (distance / time_diff) * 3.6  # Convertir m/s a km/h

# Función para dibujar las líneas de referencia
def draw_reference_lines(frame):
    height = frame.shape[0]
    cv2.line(frame, (line_start_x, 0), (line_start_x, height), line_color, line_thickness)  # Línea de inicio
    cv2.line(frame, (line_end_x, 0), (line_end_x, height), line_color, line_thickness)      # Línea de final

# Función para verificar si un vehículo cruza la línea
def crosses_line(prev_pos, curr_pos, line_x):
    px, _ = prev_pos
    cx, _ = curr_pos

    # Verificar si el vehículo cruzó la línea
    if (px < line_x and cx >= line_x) or (px > line_x and cx <= line_x):
        return True
    return False

# Captura de video
video_path = r"xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # Cambia esto por la ruta de tu video
cap = cv2.VideoCapture(video_path)

# Verificar si el video se abrió correctamente
if not cap.isOpened():
    print("Error al abrir el video.")
    exit()

# Configuración del video de salida
output_path = "output_video.mp4"
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Rotar el frame si es necesario (corregir video al revés)
    #frame = cv2.rotate(frame, cv2.ROTATE_180)  # Cambia esto según la orientación de tu video

    # Detección y seguimiento con YOLOv8
    results = model.track(frame, persist=True, tracker="bytetrack.yaml", conf=0.1)  # Ajustar confianza

    # Dibujar las líneas de referencia
    draw_reference_lines(frame)

    # Procesar cada vehículo detectado
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Obtener las coordenadas de los cuadros delimitadores
        track_ids = result.boxes.id.cpu().numpy() if result.boxes.id is not None else []  # Obtener los IDs de seguimiento
        class_ids = result.boxes.cls.cpu().numpy()  # Obtener los IDs de las clases detectadas

        for box, track_id, class_id in zip(boxes, track_ids, class_ids):
            x1, y1, x2, y2 = box
            track_id = int(track_id)
            class_name = model.names[int(class_id)]  # Obtener el nombre de la clase

            # Dibujar el cuadro alrededor del vehículo
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {track_id} ({class_name})", (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Calcular el centro del vehículo
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)

            # Verificar si el vehículo cruzó la línea de inicio
            if track_id in vehicle_times:
                prev_pos = vehicle_times[track_id][-1][1]
                if crosses_line(prev_pos, (center_x, center_y), line_start_x):
                    # Guardar el tiempo de cruce de la línea de inicio
                    vehicle_times[track_id].append((time.time(), (center_x, center_y)))
                elif crosses_line(prev_pos, (center_x, center_y), line_end_x):
                    # Calcular el tiempo de cruce entre las líneas
                    if len(vehicle_times[track_id]) >= 2:
                        start_time = vehicle_times[track_id][-2][0]
                        end_time = time.time()
                        time_diff = end_time - start_time
                        speed = calculate_speed(distance_between_lines, time_diff)

                        # Mostrar la velocidad en el frame
                        cv2.putText(frame, f"Speed: {speed:.2f} km/h", (int(x1), int(y1) - 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                        # Guardar el resultado en el DataFrame
                        results_df = pd.concat([results_df, pd.DataFrame({
                            "ID": [track_id],
                            "Speed (km/h)": [speed],
                            "Class": [class_name]
                        })], ignore_index=True)
            else:
                # Guardar la posición inicial del vehículo
                vehicle_times[track_id].append((time.time(), (center_x, center_y)))

    # Guardar el frame en el video de salida
    out.write(frame)

    # Mostrar el frame
    cv2.imshow("Frame", frame)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# Guardar el DataFrame en un archivo CSV
results_df.to_csv("vehicle_speeds.csv", index=False)
print("Resultados guardados en 'vehicle_speeds.csv'.")

# Calcular el promedio de velocidades por ID
#average_speeds = results_df.groupby("ID")["Speed (km/h)"].mean().reset_index()
#average_speeds.columns = ["ID", "Average Speed (km/h)"]
#average_speeds.to_csv("average_speeds.csv", index=False)

# Calcular el promedio de velocidades por ID
average_speeds = results_df.groupby("ID").agg({
    "Speed (km/h)": "mean",  # Promedio de velocidad
    "Class": lambda x: x.mode()[0]  # Clase más frecuente
}).reset_index()

# Renombrar las columnas
average_speeds.columns = ["ID", "Average Speed (km/h)", "Most Frequent Class"]

# Guardar el DataFrame en un archivo CSV
average_speeds.to_csv("average_speeds.csv", index=False)
print("Resultados guardados en 'average_speeds.csv'.")


0: 384x640 1 person, 2 cars, 22.1ms
Speed: 3.0ms preprocess, 22.1ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 26.5ms
Speed: 3.6ms preprocess, 26.5ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 25.5ms
Speed: 2.5ms preprocess, 25.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 27.5ms
Speed: 3.5ms preprocess, 27.5ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 29.7ms
Speed: 7.5ms preprocess, 29.7ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 airplane, 24.5ms
Speed: 2.0ms preprocess, 24.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 airplane, 28.2ms
Speed: 3.0ms preprocess, 28.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 airplane, 27.0ms
Speed: 2.5ms preprocess, 27.0ms inference, 2.9ms postprocess per 

C:\Users\joqui\AppData\Local\Temp\ipykernel_5268\4007381129.py:119: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({


0: 384x640 3 cars, 26.0ms
Speed: 2.0ms preprocess, 26.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 21.3ms
Speed: 5.0ms preprocess, 21.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 21.2ms
Speed: 2.0ms preprocess, 21.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 21.9ms
Speed: 2.5ms preprocess, 21.9ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 21.2ms
Speed: 5.5ms preprocess, 21.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 21.4ms
Speed: 3.0ms preprocess, 21.4ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 25.6ms
Speed: 2.7ms preprocess, 25.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 2 airplanes, 22.4ms
Speed: 5.5ms preprocess, 22.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)